## How to visualise data on a map if there is no latitude and longitude provided?


Let's use the data from the [WHO Global Health Observatory](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/3685) to figure out what countries have the highest prevalence rates of intimate partner violence among women aged 15-49. Consider two indicators: 

- Intimate partner violence prevalence among ever partnered women in their lifetime (%)
- Intimate partner violence prevalence among ever partnered women in the previous 12 months (%) (latest available year: 2018)


In most cases, the only way is to use a geocoder. It takes strings as input, such as addresses, and returns georeferenced locations. However,there is no free lunch. The names of the places can correspond to more than one location, such as Georgia (USA Federal State and country located in the Caucasus), and territories with controversial international recognition (Kosovo under UNSCR 1244, Palestinian territories, Hong Kong...) are hardly ever decoded correctly. What to do then? In general, providing more information to the geocoder ensures a better result. In the Georgia case, adding subregions to the query works.

In some lucky cases (as the one following), when the data have no geometry but the country codes ([ISO](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes)), we can use `plotly`, a fancy library that creates great interactive maps.


Let's begin by importing the required modules:

* Pandas for handling tabular data
* Plotly for plotting interactive maps based on ISO codes

In [1]:
# Import required modules
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Read data
df12M = pd.read_csv("https://raw.githubusercontent.com/cecilia-sartori/gs-exam/main/notebooks/data/WHO_vaw_previous12months.csv")
dfLFT = pd.read_csv('https://raw.githubusercontent.com/cecilia-sartori/gs-exam/main/notebooks/data/WHO_vaw_lifetime.csv')

In [3]:
# Merge the two indicators
df = pd.merge(df12M, dfLFT, on= 'SpatialDimValueCode', how='outer',suffixes=('_last12M','_lifetime'))
df.head(3)

,IndicatorCode_last12M,Indicator_last12M,ValueType_last12M,ParentLocationCode_last12M,ParentLocation_last12M,Location type_last12M,SpatialDimValueCode,Location_last12M,Period type_last12M,Period_last12M,...,FactValueUoM_lifetime,FactValueNumericLowPrefix_lifetime,FactValueNumericLow_lifetime,FactValueNumericHighPrefix_lifetime,FactValueNumericHigh_lifetime,Value_lifetime,FactValueTranslationID_lifetime,FactComments_lifetime,Language_lifetime,DateModified_lifetime
0,SDGIPV12M,Proportion of ever-partnered women and girls a...,numeric,AMR,Americas,Country,DOM,Dominican Republic,Year,2018.0,...,NaN,NaN,29.0,NaN,13.0,19 [29 – 13],NaN,NaN,EN,2021-05-05T21:00:00.000Z
1,SDGIPV12M,Proportion of ever-partnered women and girls a...,numeric,AMR,Americas,Country,MEX,Mexico,Year,2018.0,...,NaN,NaN,35.0,NaN,16.0,24 [35 – 16],NaN,NaN,EN,2021-05-05T21:00:00.000Z
2,SDGIPV12M,Proportion of ever-partnered women and girls a...,numeric,WPR,Western Pacific,Country,VNM,Viet Nam,Year,2018.0,...,NaN,NaN,38.0,NaN,15.0,25 [38 – 15],NaN,NaN,EN,2021-05-05T21:00:00.000Z


Create the Choropleth Map: 

In [4]:
fig = px.choropleth(df, locations= 'SpatialDimValueCode', color = 'FactValueNumeric_lifetime',
                    hover_name='Location_lifetime',
                    hover_data={'FactValueNumeric_lifetime' : True ,
                                'FactValueNumeric_last12M' : True,
                                'SpatialDimValueCode' : False
                               },
                    labels={'FactValueNumeric_lifetime':'Lifetime (%)',
                            'FactValueNumeric_last12M' : 'Past Year (%)'
                           }
                    )

fig.update_layout(
    title = go.layout.Title(
        text = 'Women subjected to violence by an intimate partner <br> at least once in their lifetime (%)'),
        title_x=0.5,
        
    hoverlabel=dict(
        bgcolor="white",
        font_size=14,
        font_family="Rockwell"
    ),
    
    geo = go.layout.Geo(
        showframe = True,
        showcountries=True, countrycolor = "white",
        showocean = True, oceancolor = '#c9d2e0',
        coastlinecolor = "white",
        projection_type = 'natural earth'),

)

config = dict({'displayModeBar': False})
fig.show(config=config)

#fig.write_html("output/VAW_in_World.html",config=config)

States are correctly displayed and data easily turned into informative interactive conten.